# PCA(主成分分析: Principal Component Analysis)
- 高次元から低次元へと変換するアルゴリズム
- 変換して低次元にするため、全ての特徴量が必要
- なのでLassoの様な特徴量選択としての用途は限定的

分散共分散行列Sの固有ベクトルuを求めれば良い

主成分(principal)
- データを最もよく表している軸
- データが最も分散している軸
- データから軸の距離の合計が最も小さくなる軸
- 射影後の分散が最も大きくなる軸

## PCAの手順

Xと主成分の数（圧縮後の次元数）を引数にしてPCA後のXを返す関数を作成

1. 平均0, 標準偏差1に標準化
2. Xの分散共分散行列を計算
3. 固有ベクトルと固有値を求める（大きい順に第一主成分, 第二主成分...と並べる）
4. 主成分にXを変換

## PCA from scratch

### データの準備

In [40]:
import numpy as np
import pandas as pd
# サンプルデータ準備
x = np.array([3, 4, 2, 3, 4, 4, 3])
y = np.array([2, 3, 5, 5, 3, 2, 3])
X = pd.DataFrame({"x": x, "y": y})
X


,x,y
0,3,2
1,4,3
2,2,5
3,3,5
4,4,3
5,4,2
6,3,3


### 完成形

In [42]:
import numpy as np
import pandas as pd
def my_pca(X, num_components):
    # step1. 標準化
    X_meaned = X - np.mean(X, axis=0)
    X_scaled = X_meaned / np.std(X_meaned, axis=0)
    
    # step2. 分散共分散行列
    cov_mat = np.cov(X_scaled, rowvar=False)
    
    # step3. 固有ベクトルと固有値
    eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_evalues = eigen_values[sorted_index]
    sorted_evectors = eigen_vectors[:, sorted_index]
    subset_evectors = sorted_evectors[:, :num_components]
    
    # step4. 変換
    X_reduced = np.dot(X_scaled, subset_evectors)
    
    return X_reduced


my_pca(X, 1)


array([[-0.49467432],
       [-0.89576549],
       [ 2.34350404],
       [ 1.33314107],
       [-0.89576549],
       [-1.50503729],
       [ 0.11459748]])

### ステップ1 標準化

X_scaledの作成

xとyそれぞれで標準化したいため、axis=0にする。

In [3]:
X_meaned = X - np.mean(X, axis=0)
X_scaled = X_meaned / np.std(X_meaned, axis=0)
X_scaled

,x,y
0,-0.408248,-1.107823
1,1.020621,-0.246183
2,-1.837117,1.477098
3,-0.408248,1.477098
4,1.020621,-0.246183
5,1.020621,-1.107823
6,-0.408248,-0.246183


### ステップ2 分散共分散行列

cov_matの作成

今回、それぞれの特徴量を表しているのは列なので、rowvar=Falseにする。

In [5]:
cov_mat = np.cov(X_scaled, rowvar=False)
cov_mat

array([[ 1.16666667, -0.73284007],
       [-0.73284007,  1.16666667]])

### ステップ3 固有ベクトルと固有値
1. eigen_values, eigen_vectorsの作成
2. eigen_valuesを大きい順に並べ替え、その順番でeigen_vectorsも並べ替える
3. subset_evectorsの作成

2において、np.linalg.eigh()のdocstringを見るとeigen_vectorsはカラムごとに固有ベクトルが入ってることがわかるので、ソートは列で行う必要がある。

In [28]:
# 1. eigen_values, eigen_vectorsの作成
# 固有値と、固有ベクトルを返す
eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)
eigen_values, eigen_vectors

(array([0.43382659, 1.89950674]),
 array([[-0.70710678, -0.70710678],
        [-0.70710678,  0.70710678]]))

In [26]:
# 2. eigen_valuesを大きい順に並べ替え、その順番でeigen_vectorsも並べ替える
# 小さい順に並べた時のインデックス
sorted_index = np.argsort(eigen_values)[::-1]
# 固有値と固有ベクトルを並び替え
sorted_evalues = eigen_values[sorted_index]
# これは列でソート
sorted_evectors = eigen_vectors[:, sorted_index]

sorted_evalues, sorted_evectors

(array([1.89950674, 0.43382659]),
 array([[-0.70710678, -0.70710678],
        [ 0.70710678, -0.70710678]]))

上のeigen_values, eigen_vectorsと比べると、ちゃんと列でソートされていることがわかる。

実際に必要なのはnum_componentsの数だけなので、上からこの数だけ持ってくる。この際も、列を持ってくることに気をつける。



In [29]:
# 3. subset_evectorsの作成
# 今は仮に2と置く。
num_components = 2
subset_evectors = sorted_evectors[:, :num_components]
subset_evectors

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

### ステップ4 主成分にXを変換
X_reducedの作成

標準化されたXに対して求めた行列をかけることで、主成分の座標系に変換できる。

In [31]:
X_reduced = np.dot(X_scaled, subset_evectors)
X_reduced

array([[-0.49467432,  1.07202459],
       [-0.89576549, -0.54761018],
       [ 2.34350404,  0.25457217],
       [ 1.33314107, -0.7557908 ],
       [-0.89576549, -0.54761018],
       [-1.50503729,  0.06166162],
       [ 0.11459748,  0.46275279]])

今回はnum_componentsを2にしているので、次元圧縮はされていない。

そして、PCA from scratchの最初に記した完成形へ。

試しに動かしてみる。

In [34]:
my_pca(X, 1)

array([[-0.49467432],
       [-0.89576549],
       [ 2.34350404],
       [ 1.33314107],
       [-0.89576549],
       [-1.50503729],
       [ 0.11459748]])

実際はsklearn.decomposition.PCAを使うので1から作ることはないが、PCAの流れに関しての理解が深まったはず。

## sklearnのPCA
sklearn.decomposition.PCA
1. PCA(n_components)でインスタンス作成
    - n_components引数には使用する主成分の数を指定
2. .fit(X)でXの主成分を計算
3. .transform(X)でXを主成分の座標系に変換

事前に標準化する必要があることに注意

### データの準備(前節と同じデータ)

In [37]:
import numpy as np
import pandas as pd
# サンプルデータ準備
x = np.array([3, 4, 2, 3, 4, 4, 3])
y = np.array([2, 3, 5, 5, 3, 2, 3])
X = pd.DataFrame({"x": x, "y": y})

### 完成形

In [43]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
pca = PCA(n_components=1)
# 標準化
X = scaler.fit_transform(X)
# Xの主成分を計算し、Xを主成分の座標系に変換
pca.fit_transform(X)



array([[-0.49467432],
       [-0.89576549],
       [ 2.34350404],
       [ 1.33314107],
       [-0.89576549],
       [-1.50503729],
       [ 0.11459748]])

これは、前節の結果と一致している。

## PCAをirisデータセットで実践

## 後ほど書く予定です、、